In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [4]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20250505'
pro_file_name = 'ACO Builder MSSP Prospective Explorer v2026.3c.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.3c.txt'
pro_risk_score_file_name = 'ACO Builder MSSP Prospective Explorer v2026.3c - 2025 Risk Scores.txt'
retro_risk_score_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.3c - 2025 Risk Scores.txt'

### Pro load

In [3]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

767191 rows, 108 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_comp_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_comp_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_comp_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_comp,reg_ret_rs_esrd_legacy,reg_ret_rs_dis_legacy,reg_ret_rs_ad_legacy,reg_ret_rs_and_legacy,reg_ret_rs_comp_legacy,reg_ret_rs_esrd_demog_legacy,reg_ret_rs_dis_demog_legacy,reg_ret_rs_ad_demog_legacy,reg_ret_rs_and_demog_legacy,reg_ret_rs_comp_demog_legacy,reg_pro_rs_esrd_legacy,reg_pro_rs_dis_legacy,reg_pro_rs_ad_legacy,reg_pro_rs_and_legacy,reg_pro_rs_comp_legacy,reg_pro_rs_esrd_demog_legacy,reg_pro_rs_dis_demog_legacy,reg_pro_rs_ad_demog_legacy,reg_pro_rs_and_demog_legacy,reg_pro_rs_comp_demog_legacy,reg_ret_rs_esrd_v24,reg_ret_rs_dis_v28,reg_ret_rs_ad_v28,reg_ret_rs_and_v28,reg_ret_rs_comp_v28,reg_ret_rs_esrd_demog_v24,reg_ret_rs_dis_demog_v28,reg_ret_rs_ad_demog_v28,reg_ret_rs_and_demog_v28,reg_ret_rs_comp_demog_v28,reg_pro_rs_esrd_v24,reg_pro_rs_dis_v28,reg_pro_rs_ad_v28,reg_pro_rs_and_v28,reg_pro_rs_comp_v28,reg_pro_rs_esrd_demog_v24,reg_pro_rs_dis_demog_v28,reg_pro_rs_ad_demog_v28,reg_pro_rs_and_demog_v28,reg_pro_rs_comp_demog_v28,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_comp,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_comp,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_comp_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_comp_v28,regional_efficiency_pro_v28
0,9335446202,00004B7C4023,474454561,2017,Prospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,NaN,NaN,NaN,NaN,NaN,2.00,9.58,4.67,50.67,66.92,1.244520,1.097477,0.634322,1.140454,1.100806,1.147136,1.131453,0.710965,1.224235,1.158854,1.155495,1.094798,0.928059,1.019671,1.036390,53840.410000,17495.682672,20291.357602,14578.463983,16568.154065,0.972462,0.931282,0.925731,0.923544,0.933452,0.973108,0.972011,0.991956,0.993545,0.987002,1.002264,0.934378,0.968965,0.961781,0.966173,0.948470,0.955312,0.986022,1.024503,0.998503,0.967368,0.942044,0.923903,0.946047,0.947263,0.986309,0.965101,1.002540,0.993990,0.989720,0.994868,0.940101,0.958124,0.980868,0.975940,0.953747,0.951744,0.994383,1.022439,0.998245,76199.31,11417.42,17473.23,10114.67,12826.223939,1.172081,74205.03,11100.84,17415.93,9893.18,12537.943438,1.199031,76595.92,11277.70,17496.22,9867.66,12335.685910,1.145185,74775.08,11028.56,17604.74,9694.96,12116.147767,1.165935
1,9335446202,00004B7C4023,474454561,2018,Prospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,NaN,NaN,NaN,NaN,NaN,1.00,13.17,11.17,95.50,120.84,0.774159,1.739319,1.081147,1.094375,1.149336,0.773194,1.639770,1.081408,1.121313,1.158229,1.220669,0.989316,0.941619,1.106579,1.071358,57672.290000,15824.800304,29136.470904,15076.135079,16809.915508,1.003112,0.946066,0.952857,0.926226,0.936279,0.958116,0.967218,0.977010,0.991929,0.985400,0.988578,0.960039,0.975052,0.966046,0.967905,0.960439,0.951587,0.973542,1.017839,1.002057,0.997572,0.942557,0.954862,0.946195,0.949966,0.957005,0.961410,0.984501,0.992555,0.986081,0.982251,0.944869,0.975444,0.982915,0.977946,0.961162,0.948420,0.982025,1.016601,1.001752,80719.75,11352.80,17563.18,10531.40,11658.305216,1.242154,77416.17,10579.37,16641.19,10319.72,11276.670921,1.284192,81168.02,11394.17,17519.19,10303.92,11473.197348,1.250927,77914.81,10747.91,16622.29,10137.23,11154.150599,1.286708
2,9335446202,00004B7C4023,474454561,2019

In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_table_name = f'milliman_pro_{current_folder}'
pro_schema = 'raw'
print(f'select * from {pro_schema}.{pro_table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_pro_20250505


In [5]:
pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

767191

In [6]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

741322 rows, 108 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,spec_count,np_count,cns_count,pcp_count,pa_count,other_count,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_comp_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_comp_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_comp_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_comp,reg_ret_rs_esrd_legacy,reg_ret_rs_dis_legacy,reg_ret_rs_ad_legacy,reg_ret_rs_and_legacy,reg_ret_rs_comp_legacy,reg_ret_rs_esrd_demog_legacy,reg_ret_rs_dis_demog_legacy,reg_ret_rs_ad_demog_legacy,reg_ret_rs_and_demog_legacy,reg_ret_rs_comp_demog_legacy,reg_pro_rs_esrd_legacy,reg_pro_rs_dis_legacy,reg_pro_rs_ad_legacy,reg_pro_rs_and_legacy,reg_pro_rs_comp_legacy,reg_pro_rs_esrd_demog_legacy,reg_pro_rs_dis_demog_legacy,reg_pro_rs_ad_demog_legacy,reg_pro_rs_and_demog_legacy,reg_pro_rs_comp_demog_legacy,reg_ret_rs_esrd_v24,reg_ret_rs_dis_v28,reg_ret_rs_ad_v28,reg_ret_rs_and_v28,reg_ret_rs_comp_v28,reg_ret_rs_esrd_demog_v24,reg_ret_rs_dis_demog_v28,reg_ret_rs_ad_demog_v28,reg_ret_rs_and_demog_v28,reg_ret_rs_comp_demog_v28,reg_pro_rs_esrd_v24,reg_pro_rs_dis_v28,reg_pro_rs_ad_v28,reg_pro_rs_and_v28,reg_pro_rs_comp_v28,reg_pro_rs_esrd_demog_v24,reg_pro_rs_dis_demog_v28,reg_pro_rs_ad_demog_v28,reg_pro_rs_and_demog_v28,reg_pro_rs_comp_demog_v28,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_comp,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_comp,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_comp_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_comp_v28,regional_efficiency_pro_v28
0,9335446202,00004B7C4023,474454561,2016,Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,15.42,8.08,71.00,96.50,1.089910,0.898524,0.743550,1.036215,0.977911,1.089910,0.898524,0.743550,1.036215,0.977911,1.190897,0.965628,1.012803,1.066019,1.051797,56585.55,16632.108949,30532.120050,15823.393239,18029.001762,0.960439,0.937959,0.927604,0.919517,0.928027,0.966233,0.970496,0.983860,1.002979,0.991670,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.960439,0.937959,0.927604,0.919517,0.928027,0.966233,0.970496,0.983860,1.002979,0.991670,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,69903.78,10551.15,17280.64,10077.21,11962.442846,1.521096,0,0,0,0,0,0,69903.78,10551.15,17280.64,10077.21,11962.442846,1.521096,0,0,0,0,0,0
1,9335446202,00004B7C4023,474454561,2017,Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20.92,12.58,142.00,176.50,0.858446,1.420928,0.927867,1.224410,1.231010,0.833297,1.479486,0.854375,1.310509,1.303791,1.221986,1.005453,1.055643,1.181950,1.152114,35210.18,22706.062141,15244.518283,20229.229718,20252.394674,0.972935,0.907434,0.929656,0.929694,0.928563,0.968670,0.973847,0.996632,0.996751,0.992896,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.966580,0.923940,0.934157,0.951709,0.946892,0.981385,0.967810,1.004986,0.996953,0.993695,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,72590.94,11490.63,17331.58,10090.22,10922.186167,1.514295,0,0,0,0,0,0,73068.24,11277.78,17238.75,9852.70,10623.265991,1.471771,0,0,0,0,0,0
2,9335446202,00004B7C4023,474454561,2018,Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,23.75,12.92,123.58,161.25,0.774159,1.281724,1.480878,1.141343,1.194690,0.773194,1.148570,1.396657,1.157925,1.177394,1.220669,0.976569,1.071435,1.156558,1.121600,57672.29,19334.744421,28502.956656,18369.726412,19567.512930,1

In [7]:
ret_table_name = f'milliman_ret_{current_folder}'
ret_schema = 'raw'
print(f'select * from {ret_schema}.{ret_table_name}')

select * from raw.milliman_ret_20250505


In [8]:
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # if_exists='replace', 

741322

## generic loading

In [5]:
pro_risk_score_file_name = 'ACO Builder MSSP Prospective Explorer v2026.3c - 2025 Risk Scores.txt'
retro_risk_score_file_name = 'ACO Builder MSSP Retrospective Explorer v2026.3c - 2025 Risk Scores.txt'
config = [
    (pro_risk_score_file_name, f'milliman_pro_risk_{current_folder}'),
    (retro_risk_score_file_name, f'milliman_ret_risk_{current_folder}'),
]

In [9]:
conn = cb_utils.get_engine(source='msh_analytics')
querries = []
for (file_name, table_name) in config:
    df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
    df = cb_utils.df_format_columns(df)
    print(f'{df.shape[0]} rows, {df.shape[1]} columns')
    display(df.head())
    
    schema = 'raw'
    print(f'select * from {schema}.{table_name}')
    querries.append(f'select * from {schema}.{table_name}')
    
    df.to_sql(table_name, conn, schema='raw', index=False, method='multi', chunksize=1000, if_exists='replace') # if_exists='replace', 

72467 rows, 26 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28
0,9335446202,00004B7C4023,474454561,2025,Prospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,0.0,8.00,1.15,74.0,83.15,NaN,1.298024,0.764234,0.915905,NaN,1.443064,0.704274,0.921253
1,5294898292,00019072C86C,263692916,2025,Prospective,CHICAGO SPORTS AND SPINE LLC,NaN,NaN,Illinois,16984.0,IL-16984,"Chicago-Naperville-Evanston, IL",60605.0,0.0,1.18,1.15,18.0,20.33,NaN,0.434621,3.158932,0.960518,NaN,0.449224,3.050175,0.815552
2,Not Available,0005468C2905,330219886,2025,Prospective,CHUNG BUI,Family Choice ACO LLC,NaN,California,11244.0,CA-11244,"Anaheim-Santa Ana-Irvine, CA",92683.0,0.0,0.00,85.00,1.2,86.20,NaN,NaN,0.871947,0.503288,NaN,NaN,0.859472,0.528512
3,446434757,00062442CB01,113492373,2025,Prospective,ANDREW GOLDFARB MD PC,NaN,NaN,New York,35004.0,NY-35004,"Nassau County-Suffolk County, NY",11040.0,0.0,0.00,1.15,65.0,66.15,NaN,NaN,0.909085,0.841008,NaN,NaN,0.779431,0.880912
4,3971678558,0006DCCB459E,742797745,2025,Prospective,WELLMED MEDICAL MANAGEMENT OF FLORIDA INC,WellMed Tampa/Orlando Medicare ACO LLC,NaN,Florida,38940.0,FL-38940,"Port St. Lucie, FL",34994.0,42.0,235.00,191.50,7324.5,7793.00,1.13338,1.086735,1.177661,1.261128,1.13338,0.969476,1.089179,1.169931


select * from raw.milliman_pro_risk_20250505
71311 rows, 26 columns


,pac_id,deid_tin,tin_public,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28
0,9335446202,00004B7C4023,474454561,2025,Retrospective,RINDFLEISCH FAMILY PRACTICE PLLC,NaN,"PEARL PRIMARY CARE NETWORK, LLC",Idaho,26820.0,ID-26820,"Idaho Falls, ID",83404.0,0.0,9.00,1.14,72.00,82.14,NaN,1.065932,0.763104,1.129820,NaN,1.139311,0.650619,1.124145
1,5294898292,00019072C86C,263692916,2025,Retrospective,CHICAGO SPORTS AND SPINE LLC,NaN,NaN,Illinois,16984.0,IL-16984,"Chicago-Naperville-Evanston, IL",60605.0,0.0,1.18,1.14,15.00,17.32,NaN,1.385629,3.678623,0.767564,NaN,1.458054,2.869274,0.648044
2,Not Available,0005468C2905,330219886,2025,Retrospective,CHUNG BUI,Family Choice ACO LLC,NaN,California,11244.0,CA-11244,"Anaheim-Santa Ana-Irvine, CA",92683.0,0.0,0.00,40.00,1.21,41.21,NaN,NaN,0.886240,0.535467,NaN,NaN,0.913328,0.502371
3,446434757,00062442CB01,113492373,2025,Retrospective,ANDREW GOLDFARB MD PC,NaN,NaN,New York,35004.0,NY-35004,"Nassau County-Suffolk County, NY",11040.0,0.0,0.00,1.14,62.00,63.14,NaN,NaN,1.443722,0.957265,NaN,NaN,1.009755,0.990324
4,3971678558,0006DCCB459E,742797745,2025,Retrospective,WELLMED MEDICAL MANAGEMENT OF FLORIDA INC,WellMed Tampa/Orlando Medicare ACO LLC,NaN,Florida,38940.0,FL-38940,"Port St. Lucie, FL",34994.0,40.0,238.00,184.50,7084.50,7547.00,1.098046,1.102582,1.051921,1.158040,1.098046,0.996534,0.968263,1.095377


select * from raw.milliman_ret_risk_20250505
